# Indicadores de Trazabilidad 



## Importación de datos, librerias

In [1]:
# Importa librerias necesarias para el análisis

import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
# Importa datos de población regional y crea DF


df_poblacion = pd.read_csv('data/aysen_population.csv') #Archivo ubicado dentro de la carpeta data

df_poblacion.head(2)

,Unnamed: 0,orden,nombre_region,codigo_region,nombre_provincia,codigo_provincia,nombre_comuna,codigo_comuna,edad_hombres,edad_mujeres,total
0,0,7172,Región de Aysén del General Carlos Ibáñez del ...,11,Coyhaique,111,Coyhaique,11101,28763,29055,57818
1,1,7194,Región de Aysén del General Carlos Ibáñez del ...,11,Coyhaique,111,Lago Verde,11102,494,358,852


In [3]:
# Importa datos de comunas corregidas por establecimiento de salud

df_establecimiento = pd.read_excel('data/establecimientos_salud_dict.xlsx') #Archivo ubicado dentro de la carpeta data


df_establecimiento.head()

,establecimiento_salud,comuna_seremi_xi,provincia_seremi_xi
0,Actividades gestionadas por la Dirección del S...,Coyhaique,Coyhaique
1,Centro Clínico Militar Coyhaique,Coyhaique,Coyhaique
2,Centro Comunitario de Salud Familiar Villa Ma...,Aysen,Aysen
3,Centro Comunitario de Salud Familiar Alejandro...,Coyhaique,Coyhaique
4,Centro Comunitario de Salud Familiar Puerto Ch...,Aysen,Aysen


In [4]:
# Importa datos desde la base de datos recivida via FTP
## PRIMERO: Guardar archivo dentro de la carpeta data
## SEGUNDO: No cambiar nombre ni formato del archivo recibido
## TERCERO: RECORDAR CAMBIAR FECHA EN EL ARCHIVO A IMPORTAR A CONTINUACION, i.e: 20210829 para este domingo 29 de Agosto 2021

df =  pd.read_csv('data/20210831_Región de Aysén del General Carlos Ibáñez del Campo_notificaciones_0600.csv', 
                   sep=';', # CSV con separador ''~''
                   low_memory=False
                  )

In [5]:
df_trazabilidad =  pd.read_csv('data/20210831_Región de Aysén del General Carlos Ibáñez del Campo_Seguimiento.csv', 
                   sep=';', # CSV con separador ''~''
                   low_memory=False
                  )

##  limpieza y ordenamiento 

In [6]:
# Une datos con diccionario comunal

dff = df.merge(df_establecimiento, how='left')

In [7]:
# Integra datos poblacionales

df_merged = dff.merge(df_poblacion.drop(columns= ['Unnamed: 0','orden','nombre_region', 'codigo_region', 'nombre_provincia','codigo_provincia']), left_on='comuna_seremi_xi', right_on='nombre_comuna', how='left' )

In [8]:
# Filtra datos obtenidos solo para pacientes con residencia en la Región de Aysén

data_aysen = df_merged[df_merged['region_residencia'] == 'Región de Aysén del General Carlos Ibáñez del Campo']

data_aysen.head(1)

,id_formulario_eno,numero_folio,id_enfermedad_eno,enfermedad_notificada,semana_epidemiologica,fecha_notificacion,etapa_clinica,establecimiento_salud,region,seremi,...,ningun_factor_riesgo,test_antigeno_sars_positivo,tac_torax_covid,comuna_seremi_xi,provincia_seremi_xi,nombre_comuna,codigo_comuna,edad_hombres,edad_mujeres,total
0,100166,50166,50,Coronavirus (COVID-19),11,2020-03-12,DESCARTADA,Hospital Regional (Coihaique),Región de Aysén del General Carlos Ibáñez del ...,SEREMI De Aisén del General Carlos Ibañez del ...,...,NaN,NaN,NaN,Coyhaique,Coyhaique,Coyhaique,11101.0,28763.0,29055.0,57818.0


In [10]:
data_aysen_filtered = data_aysen[(data_aysen['estado_caso']!='No validada')&(
                          data_aysen['vigente_no_eliminado']=='t')&(
                          data_aysen['seremi']=='SEREMI De Aisén del General Carlos Ibañez del Campo')
                       ].rename(columns={'edad_hombres':'pop_hombres',
                                         'edad_mujeres':'pop_mujeres',
                                         'total':'pop_total'})

data_aysen_filtered.sample(1)

,id_formulario_eno,numero_folio,id_enfermedad_eno,enfermedad_notificada,semana_epidemiologica,fecha_notificacion,etapa_clinica,establecimiento_salud,region,seremi,...,ningun_factor_riesgo,test_antigeno_sars_positivo,tac_torax_covid,comuna_seremi_xi,provincia_seremi_xi,nombre_comuna,codigo_comuna,pop_hombres,pop_mujeres,pop_total
5173,1643958,1593958,50,Coronavirus (COVID-19),32,2020-08-03,DESCARTADA,Hospital de Puerto Aysén,Región de Aysén del General Carlos Ibáñez del ...,SEREMI De Aisén del General Carlos Ibañez del ...,...,NaN,NaN,NaN,Aysen,Aysen,Aysen,11201.0,12719.0,11240.0,23959.0


In [11]:
# Transforma Fechas a formato DateTime 

data_aysen_filtered['fecha_notificacion'] = pd.to_datetime(data_aysen_filtered['fecha_notificacion'], errors = 'coerce') 

data_aysen_filtered['fecha_toma_muestra_1'] = pd.to_datetime(data_aysen_filtered['fecha_toma_muestra_1'], errors = 'coerce')

data_aysen_filtered['fecha_toma_muestra_2'] = pd.to_datetime(data_aysen_filtered['fecha_toma_muestra_2'], errors = 'coerce')

data_aysen_filtered['fecha_toma_muestra_3'] = pd.to_datetime(data_aysen_filtered['fecha_toma_muestra_3'], errors = 'coerce')

data_aysen_filtered['fecha_resultado_pcr_1'] = pd.to_datetime(data_aysen_filtered['fecha_resultado_pcr_1'], errors = 'coerce')

data_aysen_filtered['fecha_resultado_pcr_2'] = pd.to_datetime(data_aysen_filtered['fecha_resultado_pcr_2'], errors = 'coerce')

data_aysen_filtered['fecha_resultado_pcr_3'] = pd.to_datetime(data_aysen_filtered['fecha_resultado_pcr_3'], errors = 'coerce')

In [12]:
# Muestra variables presente en DB en formato DF

s=pd.DataFrame(data_aysen_filtered.columns)

s[s[0]=='tipo_seguimiento']


,0


In [13]:
# Selecciona variables de importancia

data_aysen_clean =  data_aysen_filtered[[
                                'numero_folio',
                                'comuna_residencia',
                                'fecha_notificacion',
                                'semana_epidemiologica',
                                'etapa_clinica',
                                'tipo_caso_busqueda',
                                'establecimiento_salud',
                                'resultado_pcr_1',
                                'resultado_pcr_2',
                                'resultado_pcr_3',
                                'fecha_toma_muestra_1',
                                'fecha_toma_muestra_2',
                                'fecha_toma_muestra_3',
                                'fecha_resultado_pcr_1',
                                'fecha_resultado_pcr_2',
                                'fecha_resultado_pcr_3',
                                'estado_paciente',
                                'lugar_reposo',
                                'estado_caso',
                                'presentacion_clinica',
                                'contacto_caso_sospechoso',
                                'motivo_examen',
                                'hospitalizacion',
                                'trabajador_salud',
                                'caso_cerrado',
                                'fecha_primeros_sintomas',
                                'fecha_diagnostico',
                                'comuna_seremi_xi',
                                'provincia_seremi_xi',
                                'pop_hombres',
                                'pop_mujeres',
                                'pop_total'
                               ]]

In [14]:
# Agrega 53 semanas a las semanas de 2021 y crea semana_epidemiologica_21

data_aysen_clean1 = data_aysen_clean[data_aysen_clean['fecha_notificacion']<=pd.to_datetime('2021-01-02')]
data_aysen_clean2 = data_aysen_clean[data_aysen_clean['fecha_notificacion']>=pd.to_datetime('2021-01-03')]


data_aysen_clean1['semana_epidemiologica_21'] = data_aysen_clean1['semana_epidemiologica']
data_aysen_clean2['semana_epidemiologica_21'] = data_aysen_clean2['semana_epidemiologica']+53

data_aysen_clean_21 = data_aysen_clean1.append(data_aysen_clean2, ignore_index=True)


<ipython-input-14-d2f259f8832e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_aysen_clean1['semana_epidemiologica_21'] = data_aysen_clean1['semana_epidemiologica']
<ipython-input-14-d2f259f8832e>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_aysen_clean2['semana_epidemiologica_21'] = data_aysen_clean2['semana_epidemiologica']+53


In [15]:
data_aysen_clean.sample(10)

,numero_folio,comuna_residencia,fecha_notificacion,semana_epidemiologica,etapa_clinica,tipo_caso_busqueda,establecimiento_salud,resultado_pcr_1,resultado_pcr_2,resultado_pcr_3,...,hospitalizacion,trabajador_salud,caso_cerrado,fecha_primeros_sintomas,fecha_diagnostico,comuna_seremi_xi,provincia_seremi_xi,pop_hombres,pop_mujeres,pop_total
169349,16519439,Coyhaique,2021-08-30,35,DESCARTADA,Búsqueda activa,Ejercicio libre de la profesión XI región,NEGATIVO,NEGATIVO,NaN,...,NaN,NO,f,NaN,NaN,Ejercicio Libre de la Profesión,Ejercicio Libre de la Profesión,NaN,NaN,NaN
164293,16191611,Cochrane,2021-08-18,33,DESCARTADA,Búsqueda activa,Hospital Lord Cochrane,NEGATIVO,NaN,NaN,...,NaN,NO,f,NaN,2021-08-20,Cochrane,Capitan Prat,1902.0,1588.0,3490.0
165880,16294716,Aisén,2021-08-23,34,DESCARTADA,Búsqueda activa,Centro de Salud Familiar Puerto Aysen,NEGATIVO,NEGATIVO,NaN,...,NaN,NO,f,NaN,2021-08-26,Aysen,Aysen,12719.0,11240.0,23959.0
125831,13321446,Chile Chico,2021-06-06,23,DESCARTADA,Atención médica,Hospital Dr. Leopoldo Ortega R. (Chile Chico),NaN,NaN,NaN,...,NaN,SI,f,2021-06-05,2021-06-06,Chile Chico,General Carrera,2577.0,2288.0,4865.0
163571,16151139,Cisnes,2021-08-18,33,DESCARTADA,Búsqueda activa,Centro de Salud Familiar La Junta,NEGATIVO,NEGATIVO,NEGATIVO,...,NaN,NO,f,NaN,2021-08-19,Cisnes,Aysen,3922.0,2595.0,6517.0
148163,14996663,Cisnes,2021-07-15,28,DESCARTADA,Búsqueda activa,Hospital Dr. Jorge Ibar (Cisnes),NEGATIVO,NEGATIVO,NaN,...,NaN,NO,f,NaN,2021-07-17,Cisnes,Aysen,3922.0,2595.0,6517.0
44604,6258168,Coyhaique,2020-12-30,53,DESCARTADA,Búsqueda activa,Posta de Salud Rural Ñireguao,NaN,NaN,NaN,...,NaN,NO,f,NaN,NaN,Coyhaique,Coyhaique,28763.0,29055.0,57818.0
115466,12605939,Coyhaique,2021-05-23,21,PROBABLE,Atención médica,Actividades gestionadas por la Dirección del S...,NEGATIVO,NEGATIVO,NaN,...,NaN,NO,f,2021-05-14,2021-05-23,Coyhaique,Coyhaique,28763.0,29055.0,57818.0
153827,15433233,Coyhaique,2021-07-28,30,DESCARTADA,Atención médica,Hospital Regional (Coihaique),NaN,NaN,NaN,...,NaN,DESCONOCIDO,f,NaN,2021-07-28,Coyhaique,Coyhaique,28763.0,29055.0,57818.0
55269,7218236,Coyhaique,2021-01-21,3,PROBABLE,Atención médica,Actividades gestionadas por la Dirección del S...,NaN,NaN,NaN,...,NaN,NO,f,2021-01-16,2021-01-21,Coyhaique,Coyhaique,28763.0,29055.0,57818.0


In [18]:
df_seguimiento = pd.read_csv('data/20210831_Región de Aysén del General Carlos Ibáñez del Campo_Seguimiento.csv',
                            sep=';',
                             low_memory=False
                            )

In [40]:
df_seguimiento.head(3)

,n_folio,tipo_seguimiento,dia_contacto,id_enfermedad_eno,enfermedad,fecha_notificacion,id_institucion,nombre_institucion_indice,region_institucion_indice,nombre_region_institucion_indice,...,folio_se_convierte_en_caso,cumple_aislamiento_cuarentena,ultimo_establecimiento_derivacion,nombre_ultimo_establecimiento_derivacion,ultimo_seremi_derivacion,región_derivación,región_seremi,región_notificacion,region_final,TTA
0,SC14618693-18497788,contacto,2021-07-08,50,Coronavirus (COVID-19),2021-07-05,13,Ejercicio libre de la profesión RM región,13,Región Metropolitana de Santiago,...,Sin Datos,t,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Región de Aysén del General Carlos Ibáñez del ...,APS
1,14585350,caso,2021-07-12,50,Coronavirus (COVID-19),2021-07-05,200380,Centro de Salud Familiar Puerto Aysen,11,Región de Aysén del General Carlos Ibáñez del ...,...,Sin Datos,t,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Región De Aysén del General Carlos Ibañez del ...,Región de Aysén del General Carlos Ibáñez del ...,APS
2,14603344,caso,2021-07-06,50,Coronavirus (COVID-19),2021-07-05,125010,Actividades gestionadas por la Dirección del S...,11,Región de Aysén del General Carlos Ibáñez del ...,...,Sin Datos,t,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Sin Datos,Región De Aysén del General Carlos Ibañez del ...,Región de Aysén del General Carlos Ibáñez del ...,SEREMI


In [53]:
# Divide strings utilizando regex para extraer contactos estrechos

df_seguimiento['n_folio'].str.extract(r'SC[0-9]+-([0-9]+)', flags=0,expand=True).rename(columns={0:'nfolio_caso'})

,nfolio_caso
0,18497788
1,NaN
2,NaN
3,18460165
4,18641978
...,...
30959,20188901
30960,NaN
30961,NaN
30962,NaN


In [47]:
# Divide strings utilizando regex para extraer contactos estrechos

df_seguimiento['n_folio'].str.extract(r'SC([0-9]+)-', flags=0,expand=True).rename(columns={0:'nfolio_sc_contacto'})

,nfolio_sc_contacto
0,14618693
1,NaN
2,NaN
3,14574306
4,14757148
...,...
30959,16034389
30960,NaN
30961,NaN
30962,NaN


In [50]:
# Divide strings utilizando regex para extraer datos con folio de Plataforma Nacional Toma de Muestras (PNTM)

df_seguimiento['n_folio'].str.extract(r'PNTM-([0-9]+)', flags=0,expand=True).rename(columns={0:'nfolio_pntm'})

,nfolio_pntm
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
30959,NaN
30960,7016056768
30961,7015472903
30962,7015314831


In [20]:
data_aysen_clean[data_aysen_clean['numero_folio']==14832232]

,numero_folio,comuna_residencia,fecha_notificacion,semana_epidemiologica,etapa_clinica,tipo_caso_busqueda,establecimiento_salud,resultado_pcr_1,resultado_pcr_2,resultado_pcr_3,...,hospitalizacion,trabajador_salud,caso_cerrado,fecha_primeros_sintomas,fecha_diagnostico,comuna_seremi_xi,provincia_seremi_xi,pop_hombres,pop_mujeres,pop_total
145734,14832232,Coyhaique,2021-07-11,28,CONFIRMADA,Atención médica,Hospital Regional (Coihaique),NEGATIVO,POSITIVO,POSITIVO,...,NaN,NO,f,2021-07-09,2021-07-12,Coyhaique,Coyhaique,28763.0,29055.0,57818.0


# Indicador 5: Oportunidad en la investigación epidemiológica de casos

## Objetivo: Fortalecer la investigación epidemiológica de los casos

### Meta: >90%

**Periodicidad:** Semanal

**Escala Territorial:** Comunal, Regional, Nacional

**Fuente de Información:** Base EPIVIGILIA actualizada con los reportes de laboratorio al día de la determinación del indicador. Base de seguimiento

### Cálculo del indicador a nivel comunal

$\frac{No.\:casos\:nuevos\:registrados\:investigados\:dentro\:de\:48\:hrs\:por\:comuna\:1er\:establecimiento,\:semana\:x}{No.\:casos\:nuevos\:registrados\:por\:comuna\:1er\:establecimiento\:semana\:x}x100$

### Cálculo del indicador a nivel regional

$\frac{No.\:casos\:nuevos\:registrados\:investigados\:dentro\:de\:48\:hrs\:por\:región\:1er\:establecimiento,\:semana\:x}{No.\:casos\:nuevos\:registrados\:por\:región\:1er\:establecimiento\:semana\:x}x100$

**Observación**

- No se incluyen los casos nuevos no notificados
- Se considera un caso nuevo registrado a aquel caso notificado que cumple los requisitos para estar en el modulo de seguimiento
- La fecha para determinar el inicio de seguimiento corresponde a la fecha de resultado (caso confirmado) o fecha de notificación (caso probable). Para un caso confirmado, si el resultado del test es anterior a la fecha de notificación, se debe iniciar seguimiento a partir de la fecha en que el registro está disponible en la bandeja de seguimiento.
- Se cumple el indicador si, dentro de las 48 hrs. a partir de la fecha de inicio de seguimiento se tiene al menos una actividad de seguimiento exitosa (se logra ubicar al caso).
- El primer establecimiento corresponde al establecimiento dado por API FONASA. Si el registro no hace match con API FONASA, se considera el establecimiento de notificación.
- Se consideran sólo registros de casos que estén activos o egresados en la bandeja de seguimiento.

# Indicador 6: Capacidad de investigación de contactos

## Objetivos: Fortalecer la investigación epidemiológica de los contactos estrechos

### Meta: >80%

**Periodicidad:** Semanal

**Escala territorial:** Comuna, Regional, Nacional

**Fuente de información:** base EPIVIGILA actualizada con los reportes de laboratorio al día de la determinación del indicador. Base de seguimiento

**Cálculo del indicador a nivel comunal**

$\frac{No.\:casos\:nuevos\:con\:contactos\:estrechos\:por\:comuna\:1er\:establecimiento\:semana\:x}{No.\:casos\:nuevos\:por\:comuna\:1er\:establecimiento\:semana\:x}$

**Cálculo del indicador a nivel regional**

$\frac{No.\:casos\:nuevos\:con\:contactos\:estrechos\:por\:región\:1er\:establecimiento\:semana\:x}{No.\:casos\:nuevos\:por\:región\:1er\:establecimiento\:semana\:x}$

**Observaciones**

- No se incluyen los casos nuevos no notificados
- Se considera un caso nuevo registrado a aquel caso nuevo notificado que cumple los requisitos para estar en el módulo de seguimiento
- Los casos nuevos con contactos estrechos son considerados de forma dicotómica (sí o no)
- El primer establecimiento corresponde al establecimiento de notificación. Se seguirá considerando el primer establecimiento para este indicador, aún si se derivó previo a la investigación epidemiológica.
- Se consideran sólo registros activos o egresados en la bandeja de seguimiento

# Indicador 7: Capacidad de investigación de contactos

## Objetivo: Evaluar la capacidad de seguimiento de contactos

### Meta: >80%

**Periodicidad:** Semanal

**Escala territorial:** Comuna, Regional, Nacional

**Fuente de información:** base EPIVIGILA actualizada con los reportes de laboratorio al día de la determinación del indicador. Base de seguimiento.

**Cálculo del indicador a nivel comunal**

$\frac{No.\:contactos\:identificados\:investigados\:dentro\:de\:48hrs\:por\:comuna\:1er\:establecimiento\:semana\:x}{No.\:contactos\:identificados\:por\:comuna\:1er\:establecimiento\:semana\:x}x100$

**Cálculo del indicador a nivel regional**

$\frac{No.\:contactos\:identificados\:investigados\:dentro\:de\:48hrs\:por\:región\:1er\:establecimiento\:semana\:x}{No.\:contactos\:identificados\:por\:región\:1er\:establecimiento\:semana\:x}x100$

**Observaciones**

- No se incluyen los contactos nuevos no notificados
- Se considera como registro de contacto aquel que cumple las condiciones para estar en la bandeja de seguimiento
- Se consideran sólo registros de casos activos o egresados en la bandeja de seguimiento
- El primer establecimiento corresponde al establecimiento dado por API FONASA. Si el registro no hace match con API FONASA, se considera el establecimiento para este indicador, aún si se derivó previo a la investigación epidemiológica

# Indicador 8: Casos en seguimiento

## Objetivo: Evaluar la capacidad de seguimiento de casos

### Meta: >90%

**Periodicidad:** Semanal

**Escala territorial:** Comuna, Regional, Nacional

**Fuente de Información:** base EPIVIGILA actualizada con los reportes de laboratorio al día de la determinación del indicador. Base de seguimiento

### Cálculo del indicador a nivel comunal

$\frac{No.\:casos\:activos\:investigados\:con\:seguimiento\:según\:protocolo\:por\:comuna\:último\:establecimiento\:semana\:x}{No.\:casos\:nuevos\:registrados\:por\:comuna\:ultimo\:establecimiento\:semana\:x}x100$

### Cálculo del indicador a nivel regional

$\frac{No.\:casos\:activos\:investigados\:con\:seguimiento\:según\:protocolo\:por\:región\:último\:establecimiento\:semana\:x}{No.\:casos\:nuevos\:registrados\:por\:región\:ultimo\:establecimiento\:semana\:x}x100$

**Observación**

- No se incluyen los casos nuevos no notificados ("casos reportados por laboratorio")
- Se considera un caso activo como aquel caso notificado que cumple la definición de caso activo, con fecha de notificación en la semana de interés
- Se consideran sólo registros activos o egresados en la bandeja de seguimiento
- Según protocolo implica un seguimiento diario por día de seguimineto, desde el inicio de seguimiento y se valora de forma dicotómica (cumple o no cumple el protocolo de seguimiento)
- Por seguimiento diario se entiende que diariamente, se debe realizar una actividad de seguimiento (la cual puede ser exitosa o no)

# Indicador 10: Contactos con al menos 2 seguimientos dentro de su periodo de cuarentena de 14 días

## Objetivo: Evaluar la capacidad de seguimiento de contactos estrechos

### Meta: >90%

**Periodicidad:** Semanal

**Escala territorial:** Comuna, Regional, Nacional

**Fuente de información:** Base de seguimiento

### Cálculo del indicador a nivel comunal

$\frac{No.\:contactos\:que\:cumplieron\:cuarentena\:con\:al\:menos\:2\:seguimientos\:por\:comuna\:último\:establecimiento\:semana\:x}{No.\:contactos\:que\:cumplieron\:cuarentena\:por\:comuna\:ultimo\:establecimiento\:semana\:x}x100$

### Cálculo del indicador a nivel regional

$\frac{No.\:contactos\:que\:cumplieron\:cuarentena\:con\:al\:menos\:2\:seguimientos\:por\:región\:último\:establecimiento\:semana\:x}{No.\:contactos\:que\:cumplieron\:cuarentena\:por\:región\:ultimo\:establecimiento\:semana\:x}x100$

**Observación**

- El indicador se construye sobre los contactos que egresan (fecha de término de cuarentena) de seguimiento, durante la semana de interés (esto es, sólo registros egresados de la bandeja de seguimiento)
- El periodo de seguimiento para el contacto corresponde a la cantidad de días entre la 1era fecha en que el contacto está disponible en la bandeja de seguimiento hasta la última fecha fin cuarentena
- Se cumple con el indicador si en el período de seguimiento del contacto se registran al menos 2 actividades de seguimiento (las cuales pueden ser exitosas o no)
- Se excluyen los contactos que se transforman en caso y que están notificados (tiene folio asociado como caso)